# 上采样与下采样

在图像处理中，上采样（Upsampling）和下采样（Downsampling）是两种常见的图像缩放技术

- 下采样（Downsampling）: 是将图像的分辨率降低的过程。通过减少图像中的像素数量来实现
  - 下采样的方法有平均池化、最大池化、卷积等方法
- 上采样（Upsampling）: 是将图像的分辨率提高的过程。通过增加图像中的像素数量来实现
  - 上采样的方法有插值法，比如最近邻插值、双线性插值和双三次插值等，还有反卷积等方法

# 高频信息与低频信息

在图像处理中，频率通常指的是图像中像素值变化的快慢

- 高频信息: 指的是图像中像素值变化较快的部分，通常对应于图像中的边缘、纹理和细节等特征。高频信息包含了图像的细节和锐利度，因此在图像处理中，高频信息对于保持图像的清晰度和细节非常重要

- 低频信息: 指的是图像中像素值变化较慢的部分，通常对应于图像中的大面积区域和渐变等特征。低频信息包含了图像的整体结构和轮廓，因此在图像处理中，低频信息对于保持图像的整体感知和视觉效果非常重要

# 图像金字塔

参考：

[图像金字塔](https://www.opencv.org.cn/opencvdoc/2.3.2/html/doc/tutorials/imgproc/pyramids/pyramids.html)

[图像处理基础（十）拉普拉斯金字塔、压缩、图像融合](https://zhuanlan.zhihu.com/p/454085730)

尺度，顾名思义就是说图像的尺寸和分辨率。在我们进行图像处理的时候，会经常对源图像的尺寸进行放大或者缩小的变换，进而转换为我们指定尺寸的目标图像。在对图像进行放大和缩小的变换的这个过程，我们称为尺度调整

一个图像金字塔是一系列图像的集合，所有图像来源于同一张原始图像，通过梯次向下采样获得，直到达到某个终止条件才停止采样

有两种类型的图像金字塔常常出现在文献和应用中:

- 高斯金字塔(Gaussian pyramid): 用来向下采样
- 拉普拉斯金字塔(Laplacian pyramid): 用来从金字塔低层图像重建上层未采样图像

## 高斯金字塔

![](./images/高斯金字塔.png)

如图是一个高斯金字塔的示意图

- 每一层都按从下到上的次序编号，层级$(i+1)$(表示为$G_{i+1}$)尺寸小于层级$i$($G_i$)。
- 为了获取层级为$(i+1)$的金字塔图像，我们采用如下方法:
  - 将$G_i$与高斯内核卷积:
    $$\frac{1}{16}\begin{bmatrix}
    1 & 4 & 6 & 4 & 1 \\
    4 & 16 & 24 & 16 & 4 \\
    6 & 24 & 36 & 24 & 6 \\
    4 & 16 & 24 & 16 & 4 \\
    1 & 4 & 6 & 4 & 1 \\
    \end{bmatrix}$$
    这里的kernel是一个5x5的高斯滤波器，所有元素之和为256，乘以$\frac{1}{16}$是为将卷积结果归一化（之所以不乘以$\frac{1}{256}$是为了方便整数计算，在系统内部事先对系数整体缩放了16倍）
  - 将所有偶数行和列去除
- 显而易见，结果图像只有原图的四分之一。通过对输入图像$G_0$(原始图像)不停迭代以上步骤就会得到整个金字塔

高斯金字塔描述了对图像的向下采样，如果将图像变大呢?

- 首先，将图像在每个方向扩大为原来的两倍，新增的行和列以0填充(0)
- 使用先前同样的内核(乘以4)与放大后的图像卷积，获得 “新增像素” 的近似值

这两个步骤(向下和向上采样)分别通过OpenCV函数`pyrUp`和`pyrDown`实现

函数原型：

```python
dst = cv2.pyrDown(src[, dstsize[, borderType]])
dst = cv2.pyrUp(src[, dstsize[, borderType]])
```

参数说明：

- `src`: 输入图像
- `dstsize`: 输出图像的大小，默认情况下，`pyrDown`将图像尺寸减半，`pyrUp`将图像尺寸加倍，若要指定其他尺寸，可以通过该参数设置，例如`dstsize=(width, height)`
- `borderType`: 边界模式，默认值为`cv2.BORDER_DEFAULT`

In [4]:
import cv2
import numpy as np
from my_function import imshow
img=cv2.imread("./images/sunflower.jpg")
print(img.shape)
down=cv2.pyrDown(img)
up=cv2.pyrUp(img)
down_up=cv2.pyrUp(down)
imshow(down=down,up=up,down_up=down_up)

(720, 480, 3)


可以发现，当对已经下采样的图像进行上采样时，图像会变得模糊，因为在下采样过程中丢失了一些信息

## 拉普拉斯金字塔

拉普拉斯金字塔将下采样所得图像金字塔与上采样所得图像金字塔相减（同分辨率相减），从而得到一组图像

In [5]:
down_2=cv2.pyrDown(down)
up_2=cv2.pyrUp(down_2)
up_3=cv2.pyrUp(up_2)
low=(down.astype(np.float32)-up_2.astype(np.float32))
high=(img.astype(np.float32)-up_3.astype(np.float32))
# 可视化：偏移到 [0,255] 范围
low_vis=np.clip(low+128.0, 0, 255).astype(np.uint8)
high_vis=np.clip(high+128.0, 0, 255).astype(np.uint8)
imshow(high_vis=high_vis,low_vis=low_vis)